# Week 2 Homework Solution

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command mlflow --version and check the output.

What's the version that you have?

In [1]:
!mlflow --version

mlflow, version 1.26.0


#### Answer
> MLflow version: **1.26.0**

## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset. 

Download the data for January, February and March 2021 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/02-experiment-tracking/homework) to preprocess the data.

The script will

- load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
- fit a `DictVectorizer` on the training set (January 2021 data),
- save the preprocessed datasets and the model to disk

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7

In [36]:
# Get Jan - Mar 2021 Green Taxi data

!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet -P ./data
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet -P ./data
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-03.parquet -P ./data

--2022-06-02 09:24:13--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet
Connecting to 192.9.200.39:80... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1333519 (1.3M) [binary/octet-stream]
Saving to: ‘./data/green_tripdata_2021-01.parquet.1’

green_tripdata_2021 100%[===================>]   1.27M   144KB/s    in 16s     

2022-06-02 09:24:32 (84.0 KB/s) - ‘./data/green_tripdata_2021-01.parquet.1’ saved [1333519/1333519]

--2022-06-02 09:24:32--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet
Connecting to 192.9.200.39:80... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1145679 (1.1M) [binary/octet-stream]
Saving to: ‘./data/green_tripdata_2021-02.parquet.1’

green_tripdata_2021 100%[===================>]   1.09M   152KB/s    in 9.8s    

2022-06-02 09:24:43 (114 KB/s) - ‘./data/green_tripdata_2021-02.parquet.1’ saved [1145679/1145679]

--2022-06-02 09:24:43--  https://s3.amazonaws.com/nyc-tlc

In [7]:
!python preprocess_data.py --raw_data_path ./data --dest_path ./output

In [8]:
!ls ./output

dv.pkl	test.pkl  train.pkl  valid.pkl


#### Answer
> **4 files** are in the `OUTPUT_FOLDER`

## Q3. Train a model with autolog

We will train a random forest regressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. The script will load the datasets produced by the previous step, train the model on the training set and finally calculate the RMSE on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

How many parameters are automatically logged by MLflow?

* 19
* 17
* 10
* 20

In [19]:
# Modified `train.py` to enable autologging in MLflow
!cat train.py

import argparse
import os
import pickle

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import mlflow

def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def run(data_path):

    # start mlflow
    with mlflow.start_run():
        # use autolog
        mlflow.sklearn.autolog()

        X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
        X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))

        rf = RandomForestRegressor(max_depth=10, random_state=0)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_valid)

        rmse = mean_squared_error(y_valid, y_pred, squared=False)


if __name__ == '__main__':

    # Setting the tracking_uri for mlflow
    mlflow.set_tracking_uri("sqlite:///..//mlflow.db")

    # Create a new experiment for green_taxi
    mlflow.set_experiment("green-taxi_experiment")

    parser = argparse.ArgumentPa

In [20]:
!python train.py

2022/06/02 08:51:28 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi_experiment' does not exist. Creating a new experiment.
2022/06/02 08:51:28 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/06/02 08:51:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/bengsoon/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [21]:
from mlflow.tracking import MlflowClient

TRACKING_URI = "sqlite:///..//mlflow.db"

client = MlflowClient(tracking_uri=TRACKING_URI)

In [26]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/4', experiment_id='4', lifecycle_stage='active', name='test', tags={}>,
 <Experiment: artifact_location='./mlruns/5', experiment_id='5', lifecycle_stage='active', name='green-taxi_experiment', tags={}>]

In [27]:
client.search_runs(experiment_ids=5)

[<Run: data=<RunData: metrics={'training_mae': 3.9579721044559886,
  'training_mse': 32.842658732583544,
  'training_r2_score': 0.7543646399020425,
  'training_rmse': 5.73085148408014,
  'training_score': 0.7543646399020425}, params={'bootstrap': 'True',
  'ccp_alpha': '0.0',
  'criterion': 'squared_error',
  'max_depth': '10',
  'max_features': '1.0',
  'max_leaf_nodes': 'None',
  'max_samples': 'None',
  'min_impurity_decrease': '0.0',
  'min_samples_leaf': '1',
  'min_samples_split': '2',
  'min_weight_fraction_leaf': '0.0',
  'n_estimators': '100',
  'n_jobs': 'None',
  'oob_score': 'False',
  'random_state': '0',
  'verbose': '0',
  'warm_start': 'False'}, tags={'estimator_class': 'sklearn.ensemble._forest.RandomForestRegressor',
  'estimator_name': 'RandomForestRegressor',
  'mlflow.log-model.history': '[{"run_id": "bbee2bd09cd94b88855f0e2b0bb96ca3", '
                              '"artifact_path": "model", "utc_time_created": '
                              '"2022-06-02 00:51:4

In [28]:
RUN_ID = "bbee2bd09cd94b88855f0e2b0bb96ca3"

run = client.get_run(run_id=RUN_ID)

Here are all the parameters that were autologged by MLflow

In [31]:
run.data.params

{'bootstrap': 'True',
 'ccp_alpha': '0.0',
 'criterion': 'squared_error',
 'max_depth': '10',
 'max_features': '1.0',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'n_estimators': '100',
 'n_jobs': 'None',
 'oob_score': 'False',
 'random_state': '0',
 'verbose': '0',
 'warm_start': 'False'}

In [32]:
len(run.data.params)

17

#### Answer 
> There are **17 parameters** logged with `autolog`

## Q4. Launch the tracking server locally

We want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to

* launch the tracking server on your local machine
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`


In [35]:
!mlflow server --help

Usage: mlflow server [OPTIONS]

  Run the MLflow tracking server.

  The server listens on http://localhost:5000 by default and only accepts
  connections from the local machine. To let the server accept connections
  from other machines, you will need to pass ``--host 0.0.0.0`` to listen on
  all network interfaces (or a specific interface address).

Options:
  --backend-store-uri PATH     URI to which to persist experiment and run
                               data. Acceptable URIs are SQLAlchemy-compatible
                               database connection strings (e.g.
                               'sqlite:///path/to/file.db') or local
                               filesystem URIs (e.g.
                               'file:///absolute/path/to/directory'). By
                               default, data will be logged to the ./mlruns
                               directory.
  --default-artifact-root URI  Directory in which to store artifacts for any
                             

![](./img/q4_mlflow_tracking_server.png)

#### Answer
> **`--default-artifact-root `** to change the default location of the artifacts location
>  - `--serve-artifacts` is to enable the requests to be done through a proxy
>  - `--artifacts-only` is an option to configure the mlflow server to be used only for proxied artifact serving
>  - `--artifacts-destination` only applies to the tracking server that is serving through "http" or "mlflow-artifacts URI"

Reference: https://www.mlflow.org/docs/latest/tracking.html

## Q5. Tune the hyperparameters of the model

Now let's try to reduce the validation error by tuning the hyperparameters of the random forest regressor using `hyperopt`.

We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to MLflow for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open the MLflow UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Important: don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization.
* the RMSE obtained on the validation set (February 2021 data).

What's the best validation RMSE that you got?

* 6.128
* 6.628
* 7.128
* 7.628

In [39]:
# added a few lines of code to enable mlflow logging for the `objective` function
!cat hpo.py

import argparse
import os
import pickle

import mlflow
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("random-forest-hyperopt")


def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def run(data_path, num_trials):

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))

    def objective(params):

        with mlflow.start_run():
            # log the "model" tag 
            mlflow.set_tag("model", "random-forest")  

            # log all the hyperparameters that got passed into this function
            mlflow.log_params(params)

            rf = RandomForestRegressor(**params)
            rf.fit(X_train, y_train)
       

In [43]:
# run the hpo.py

!python hpo.py

2022/06/02 09:54:17 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
100%|█████████| 50/50 [07:53<00:00,  9.47s/trial, best loss: 6.6284257482044735]


In [45]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/4', experiment_id='4', lifecycle_stage='active', name='test', tags={}>,
 <Experiment: artifact_location='./mlruns/5', experiment_id='5', lifecycle_stage='active', name='green-taxi_experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/6', experiment_id='6', lifecycle_stage='active', name='random-forest-hyperopt', tags={}>]

In [51]:
best_runs = client.search_runs(
    experiment_ids=6,
    filter_string="",
    max_results=3,
    order_by=["metrics.rmse ASC"]
)

In [54]:
for run in best_runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 1f239884901c4405b4983eb624d40568, rmse: 6.6284
run id: c5f768f369974570bead026a3a3d69cc, rmse: 6.6297
run id: 3c470f375e7845e9aebe2ab61a7e8f8a, rmse: 6.6299


#### Answer
> Best Validation Run RMSE: **6.6284**

## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good so we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2021 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE.
Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right model_uri in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 6.1
* 6.55
* 7.93
* 15.1

In [56]:
# Updated register_model.py so that it selects the model with the lowest RMSE on the test set
!cat register_model.py

import argparse
import os
import pickle

import mlflow
from hyperopt import hp, space_eval
from hyperopt.pyll import scope
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

HPO_EXPERIMENT_NAME = "random-forest-hyperopt"
EXPERIMENT_NAME = "random-forest-best-models"

mlflow.set_tracking_uri("sqlite:///../mlflow.db")
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.sklearn.autolog()

SPACE = {
    'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
    'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
    'random_state': 42
}


def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def train_and_log_model(data_path, params):
    X_train, y_tr

In [57]:
!python register_model.py

2022/06/02 10:24:46 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2022/06/02 10:24:46 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/06/02 10:25:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/bengsoon/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'greentaxi_regressor'.
2022/06/02 10:26:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: greentaxi_regressor, version 1
Created version '1' of model 'greentaxi_regressor'.


In [63]:
# get run_id from version 1 of 'greentaxi_regressor'

version_1_run_id = client.get_model_version(name = "greentaxi_regressor", version = 1).run_id
run_ver1 = client.get_run(run_id=version_1_run_id)

In [71]:
print(f"Version 1 Test RMSE: {round(run_ver1.data.metrics['test_rmse'],2)}")

Version 1 Test RMSE: 6.55


#### Answer
> Version 1 Test RMSE: **6.55**